In [ ]:
pip install pandas numpy matplotlib seaborn scikit-learn xgboost lightgbm shap

In [16]:
# ==============================================================================
# Comprehensive Data Analysis Script for Q1 Paper - Regression Version
# Objective: Predict Water Usage Efficiency
# Author: AI Assistant (Adapted for Regression)
# Version: 5.6 - Wider Subplot Spacing
# ==============================================================================

# ------------------------------------------------------------------------------
# Phase 0: Initial Setup, Libraries, and Data Loading
# ------------------------------------------------------------------------------
print("Phase 0: Starting initial setup")

# --- Import essential libraries ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import joblib
import shap
from scipy import stats

# --- Import sklearn tools ---
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# --- Import regression models ---
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# --- General Settings ---
sns.set_theme(style="whitegrid")
plt.rcParams.update({
    'font.size': 7,
    'axes.titlesize': 7,
    'axes.labelsize': 7,
    'xtick.labelsize': 7,
    'ytick.labelsize': 7,
    'legend.fontsize': 7,
    'figure.titlesize': 7
})
RANDOM_STATE = 42
TARGET_VARIABLE = 'water_usage_efficiency'

# --- Define Paths ---
BASE_DIR = '/home/sajad/Sajad_test/Smart_Farming_Data_2024 _Regression/'
FILE_NAME = 'Crop_recommendationV2.csv'
DATA_PATH = os.path.join(BASE_DIR, 'dataset', FILE_NAME)

DATA_RESULTS_PATH = os.path.join(BASE_DIR, 'data_results')
COMPARE_PATH = os.path.join(BASE_DIR, 'comparison')
EDA_PATH = os.path.join(BASE_DIR, 'EDA')
MODELS_PATH = os.path.join(BASE_DIR, 'Models')
STATS_PATH = os.path.join(BASE_DIR, 'Statistical_Analysis')
BEST_MODEL_PATH = os.path.join(BASE_DIR, 'Best_Model_Analysis')

paths_to_create = [DATA_RESULTS_PATH, COMPARE_PATH, EDA_PATH, MODELS_PATH, STATS_PATH, BEST_MODEL_PATH]
for path in paths_to_create:
    os.makedirs(path, exist_ok=True)
print("All directories created or verified successfully.")

# --- Helper Functions ---
def save_plot(fig, path, filename, width_cm=15):
    """Helper function to save plots in various formats."""
    width_in = width_cm / 2.54
    aspect_ratio = fig.get_figheight() / fig.get_figwidth()
    fig.set_size_inches(width_in, width_in * aspect_ratio)
    fig.savefig(os.path.join(path, f"{filename}.svg"), format='svg', bbox_inches='tight', dpi=300)
    fig.savefig(os.path.join(path, f"{filename}.pdf"), format='pdf', bbox_inches='tight', dpi=300)
    plt.close(fig)
    print(f"Plot saved: {filename}")

def get_regression_metrics(y_true, y_pred):
    """Calculate all regression evaluation metrics."""
    metrics = {
        'R-squared (R²)': r2_score(y_true, y_pred),
        'Mean Absolute Error (MAE)': mean_absolute_error(y_true, y_pred),
        'Mean Squared Error (MSE)': mean_squared_error(y_true, y_pred),
        'Root Mean Squared Error (RMSE)': np.sqrt(mean_squared_error(y_true, y_pred)),
    }
    return metrics

def plot_actual_vs_predicted(y_true, y_pred, ax, title):
    """Plot actual vs. predicted values and annotate with regression stats."""
    ax.scatter(y_true, y_pred, alpha=0.5, edgecolors='k', s=20)
    # The red dashed line represents the ideal scenario where Predicted = Actual
    ax.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--', lw=2, label='Ideal Fit (y=x)')
    ax.set_xlabel('Actual Values')
    ax.set_ylabel('Predicted Values')
    ax.set_title(title)
    ax.grid(True)

    # Calculate and add regression equation and R²
    slope, intercept, r_value, _, _ = stats.linregress(y_true, y_pred)
    r_squared = r_value**2
    equation = f'y = {slope:.2f}x + {intercept:.2f}'
    r2_text = f'R² = {r_squared:.3f}'
    
    ax.text(0.05, 0.95, f'{equation}\n{r2_text}', transform=ax.transAxes, fontsize=7,
            verticalalignment='top', bbox=dict(boxstyle='round,pad=0.3', fc='wheat', alpha=0.5))
    ax.legend(loc='lower right')


def plot_residuals(y_true, y_pred, ax, title):
    """Plot residuals."""
    residuals = y_true - y_pred
    ax.scatter(y_pred, residuals, alpha=0.5, edgecolors='k', s=20)
    ax.axhline(y=0, color='r', linestyle='--')
    ax.set_xlabel('Predicted Values')
    ax.set_ylabel('Residuals')
    ax.set_title(title)
    ax.grid(True)

def plot_regression_seaborn(y_true, y_pred, ax, title):
    """Plot regression plot with seaborn, and annotate with R² and the regression equation."""
    sns.regplot(x=y_true, y=y_pred, ax=ax, scatter_kws={'alpha':0.3, 's':15}, line_kws={'color': 'red', 'lw': 2})
    slope, intercept, r_value, _, _ = stats.linregress(y_true, y_pred)
    r_squared = r_value**2
    equation = f'y = {slope:.2f}x + {intercept:.2f}'
    r2_text = f'R² = {r_squared:.3f}'
    ax.text(0.05, 0.95, f'{equation}\n{r2_text}', transform=ax.transAxes, fontsize=7,
            verticalalignment='top', bbox=dict(boxstyle='round,pad=0.3', fc='wheat', alpha=0.5))
    ax.set_xlabel('Actual Values')
    ax.set_ylabel('Predicted Values')
    ax.set_title(title)
    ax.grid(True)

# --- Load Data ---
try:
    df = pd.read_csv(DATA_PATH)
    print("Dataset loaded successfully.")
except FileNotFoundError:
    print(f"!!!!!!!!!! CRITICAL ERROR !!!!!!!!!!!\nDataset file not found at:\n{DATA_PATH}")
    exit()

# ------------------------------------------------------------------------------
# Phase 1: Exploratory Data Analysis (EDA)
# ------------------------------------------------------------------------------
print("\nPhase 1: Starting Exploratory Data Analysis (EDA)")
df.describe().to_excel(os.path.join(EDA_PATH, "descriptive_statistics.xlsx"))

fig, ax = plt.subplots()
sns.histplot(df[TARGET_VARIABLE], kde=True, ax=ax, color='teal')
ax.set_title(f'Distribution of Target Variable: {TARGET_VARIABLE}')
save_plot(fig, EDA_PATH, "target_variable_distribution", width_cm=12)

numerical_features_all = df.select_dtypes(include=np.number).columns.tolist()
corr_matrix = df[numerical_features_all].corr()
fig, ax = plt.subplots(figsize=(20, 17))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".1f", annot_kws={"size": 7})
ax.set_title("Feature Correlation Matrix")
save_plot(fig, EDA_PATH, 'correlation_heatmap_all_features', width_cm=30)
print("Exploratory Data Analysis completed.")

# ------------------------------------------------------------------------------
# Phase 1.5: Inferential Statistical Analysis
# ------------------------------------------------------------------------------
print("\nPhase 1.5: Starting Inferential Statistical Analysis")
if 'label' in df.columns:
    groups = [df[TARGET_VARIABLE][df['label'] == crop] for crop in df['label'].unique()]
    groups = [g for g in groups if len(g) > 1]
    if len(groups) > 1:
        f_val, p_val = stats.f_oneway(*groups)
        anova_results_df = pd.DataFrame([{'Comparison': f'Overall across all crops for {TARGET_VARIABLE}', 'F-statistic': f_val, 'p-value': p_val}])
        anova_results_df.to_excel(os.path.join(STATS_PATH, 'anova_test_results.xlsx'), index=False)
        print("Statistical analysis (ANOVA) completed and results saved.")

        fig, ax = plt.subplots(figsize=(12, 8))
        sns.boxplot(x='label', y=TARGET_VARIABLE, data=df, ax=ax, palette="viridis")
        ax.set_title(f'Comparison of "{TARGET_VARIABLE}" by Crop Type')
        plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
        fig.tight_layout()
        save_plot(fig, STATS_PATH, f'boxplot_{TARGET_VARIABLE}_by_crop', width_cm=20)
    else:
        print("Not enough groups found for ANOVA test.")
else:
    print("Column 'label' not found in the dataset for ANOVA analysis.")

# ------------------------------------------------------------------------------
# Phase 2: Data Preprocessing and Preparation
# ------------------------------------------------------------------------------
print("\nPhase 2: Starting data preprocessing")
if TARGET_VARIABLE not in df.columns:
    raise ValueError(f"Target variable '{TARGET_VARIABLE}' not found in the dataset!")

X = df.drop(TARGET_VARIABLE, axis=1)
y = df[TARGET_VARIABLE]

categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_features = X.select_dtypes(include=np.number).columns.tolist()
if 'label' not in categorical_features and 'label' in X.columns:
    categorical_features.append('label')
numerical_features = [f for f in numerical_features if f not in categorical_features]

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ],
    remainder='passthrough'
)
joblib.dump(preprocessor, os.path.join(DATA_RESULTS_PATH, 'preprocessor_regression.joblib'))

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE
)

print("Saving train and test sets for reproducibility...")
X_train.to_csv(os.path.join(DATA_RESULTS_PATH, 'X_train.csv'), index=False)
X_test.to_csv(os.path.join(DATA_RESULTS_PATH, 'X_test.csv'), index=False)
y_train.to_csv(os.path.join(DATA_RESULTS_PATH, 'y_train.csv'), index=False, header=True)
y_test.to_csv(os.path.join(DATA_RESULTS_PATH, 'y_test.csv'), index=False, header=True)
print("Data preprocessing and splitting completed. All necessary artifacts saved.")

# ------------------------------------------------------------------------------
# Phase 3: Comprehensive Comparison of Regression Models
# ------------------------------------------------------------------------------
print("\nPhase 3: Starting comprehensive model evaluation and comparison")
models = {
    'Linear Regression': LinearRegression(), 'Ridge': Ridge(random_state=RANDOM_STATE),
    'Lasso': Lasso(random_state=RANDOM_STATE), 'KNN Regressor': KNeighborsRegressor(),
    'SVR': SVR(), 'Random Forest Regressor': RandomForestRegressor(random_state=RANDOM_STATE),
    'XGBoost Regressor': XGBRegressor(random_state=RANDOM_STATE),
    'LightGBM Regressor': LGBMRegressor(random_state=RANDOM_STATE, verbosity=-1),
}

all_models_metrics = []

for name, model in models.items():
    print(f"--- Evaluating model: {name} ---")
    pipeline = Pipeline([('preprocessor', preprocessor), ('reg', model)])
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    
    model_filename = name.replace(' ', '_')
    
    predictions_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
    predictions_save_path = os.path.join(DATA_RESULTS_PATH, f"predictions_{model_filename}.csv")
    predictions_df.to_csv(predictions_save_path, index=False)
    print(f"Predictions for {name} saved to: {predictions_save_path}")

    joblib.dump(pipeline, os.path.join(MODELS_PATH, f"{model_filename}_pipeline.joblib"))

    metrics = get_regression_metrics(y_test, y_pred)
    metrics['Model'] = name
    all_models_metrics.append(metrics)
    pd.DataFrame([metrics]).to_excel(os.path.join(MODELS_PATH, f'{model_filename}_metrics.xlsx'), index=False)

    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 6))
    fig.suptitle(f'Model Evaluation: {name}')
    
    plot_actual_vs_predicted(y_test, y_pred, ax1, 'Actual vs. Predicted')
    plot_residuals(y_test, y_pred, ax2, 'Residuals Analysis')
    plot_regression_seaborn(y_test, y_pred, ax3, 'Regression Plot (Seaborn)')
    
    # MODIFICATION 4: Increased wspace to create a wider gap between plots as requested.
    # A wspace of 0.17 corresponds to an approximate 1.5 cm gap.
    fig.subplots_adjust(left=0.05, right=0.98, wspace=0.17, top=0.9, bottom=0.1)
    
    save_plot(fig, MODELS_PATH, f'evaluation_plots_{model_filename}', width_cm=30)

results_df = pd.DataFrame(all_models_metrics).set_index('Model').sort_values(by='R-squared (R²)', ascending=False)
results_df.to_excel(os.path.join(COMPARE_PATH, "all_models_metrics_comparison.xlsx"))

results_df_plot = results_df[['R-squared (R²)', 'Root Mean Squared Error (RMSE)']].reset_index()
results_df_melted = results_df_plot.melt(id_vars='Model', var_name='Metric', value_name='Score')

fig_metrics, ax_metrics = plt.subplots(figsize=(10, 7))
sns.barplot(x='Score', y='Model', hue='Metric', data=results_df_melted, ax=ax_metrics, palette='viridis', orient='h')
ax_metrics.set_title('Model Performance Comparison')
ax_metrics.set_xlabel('Score')
ax_metrics.set_ylabel('Model')
fig_metrics.tight_layout()
save_plot(fig_metrics, COMPARE_PATH, 'models_metrics_comparison_bar_chart', width_cm=20)

best_model_name = results_df.index[0]
print(f"\nBest model based on R-squared: {best_model_name}")

# ------------------------------------------------------------------------------
# Phase 4: Hyperparameter Tuning of the Best Model
# ------------------------------------------------------------------------------
print(f"\nPhase 4: Starting hyperparameter tuning for {best_model_name}")
model_base = models[best_model_name]
param_dist = {}

if 'XGBoost' in best_model_name:
    param_dist = {
        'reg__n_estimators': [100, 300, 500, 700], 'reg__learning_rate': [0.01, 0.05, 0.1],
        'reg__max_depth': [3, 5, 7, 9], 'reg__colsample_bytree': [0.7, 0.8, 0.9, 1.0],
        'reg__subsample': [0.7, 0.8, 0.9, 1.0]
    }
elif 'Random Forest' in best_model_name:
     param_dist = {
        'reg__n_estimators': [100, 200, 300, 500], 'reg__max_depth': [10, 20, 30, None],
        'reg__min_samples_split': [2, 5, 10], 'reg__min_samples_leaf': [1, 2, 4]
    }
elif 'LightGBM' in best_model_name:
    param_dist = {
        'reg__n_estimators': [100, 300, 500, 700], 'reg__learning_rate': [0.01, 0.05, 0.1],
        'reg__num_leaves': [20, 31, 40, 50], 'reg__max_depth': [-1, 5, 10],
    }
else:
    print("Hyperparameter tuning is not defined for this model.")
    param_dist = None

if param_dist:
    pipeline = Pipeline([('preprocessor', preprocessor), ('reg', model_base)])
    random_search = RandomizedSearchCV(
        estimator=pipeline, param_distributions=param_dist, n_iter=50,
        cv=KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE),
        scoring='neg_root_mean_squared_error', n_jobs=-1, random_state=RANDOM_STATE, verbose=1
    )
    random_search.fit(X_train, y_train)
    print("\nBest parameters found:", random_search.best_params_)
    final_model_pipeline = random_search.best_estimator_
else:
    final_model_pipeline = Pipeline([('preprocessor', preprocessor), ('reg', model_base)]).fit(X_train, y_train)

joblib.dump(final_model_pipeline, os.path.join(BEST_MODEL_PATH, "final_tuned_model_pipeline.joblib"))
print(f"Tuned model {best_model_name} saved in the Best_Model_Analysis folder.")

# ------------------------------------------------------------------------------
# Phase 5: In-depth Analysis of the Best Model with SHAP
# ------------------------------------------------------------------------------
print(f"\nPhase 5: Starting in-depth analysis of {best_model_name} with SHAP")

final_model = final_model_pipeline.named_steps['reg']
preprocessor_fitted = final_model_pipeline.named_steps['preprocessor']
X_test_transformed = preprocessor_fitted.transform(X_test)
feature_names_out = preprocessor_fitted.get_feature_names_out()
X_test_transformed_df = pd.DataFrame(X_test_transformed, columns=feature_names_out, index=X_test.index)

try:
    if isinstance(final_model, (RandomForestRegressor, XGBRegressor, LGBMRegressor)):
        explainer = shap.TreeExplainer(final_model)
    else:
        X_train_transformed_summary = shap.sample(preprocessor_fitted.transform(X_train), 100)
        explainer = shap.KernelExplainer(final_model.predict, X_train_transformed_summary)

    shap_values = explainer(X_test_transformed_df)
    
    shap_data_to_save = {
        'shap_values': shap_values.values,
        'base_values': shap_values.base_values,
        'data': X_test_transformed_df
    }
    joblib.dump(shap_data_to_save, os.path.join(DATA_RESULTS_PATH, 'shap_data.joblib'))
    print("SHAP values and transformed test data saved for reproducibility.")

    shap.summary_plot(shap_values, X_test_transformed_df, plot_type="bar", show=False, max_display=20)
    fig_shap_bar = plt.gcf()
    plt.title(f"SHAP Feature Importance (Model: {best_model_name})")
    save_plot(fig_shap_bar, BEST_MODEL_PATH, "shap_summary_bar_plot", width_cm=20)
    
    shap.summary_plot(shap_values, X_test_transformed_df, show=False, max_display=20)
    fig_shap_dot = plt.gcf()
    plt.title(f"SHAP Summary of Feature Effects (Model: {best_model_name})")
    save_plot(fig_shap_dot, BEST_MODEL_PATH, "shap_summary_dot_plot", width_cm=20)
    
    top_features = X_test_transformed_df.columns[np.argsort(np.abs(shap_values.values).mean(0))][::-1]
    
    print("Creating SHAP dependence plots for top features...")
    for feature in top_features[:5]:
        fig_dep, ax_dep = plt.subplots()
        shap.dependence_plot(feature, shap_values.values, X_test_transformed_df, ax=ax_dep, show=False)
        ax_dep.set_title(f'SHAP Dependence Plot for Feature: {feature}')
        save_plot(fig_dep, BEST_MODEL_PATH, f"shap_dependence_plot_{feature.replace('/', '_').replace('<', '_lt_')}", width_cm=15)

except Exception as e:
    print(f"SHAP analysis failed with error: {e}")

print("\n" + "="*80 + "\nMain script executed successfully! All results and artifacts are saved.\n" + "="*80)

Phase 0: Starting initial setup
All directories created or verified successfully.
Dataset loaded successfully.

Phase 1: Starting Exploratory Data Analysis (EDA)
Plot saved: target_variable_distribution


Plot saved: correlation_heatmap_all_features
Exploratory Data Analysis completed.

Phase 1.5: Starting Inferential Statistical Analysis
Statistical analysis (ANOVA) completed and results saved.


/tmp/ipykernel_3914310/132650710.py:176: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='label', y=TARGET_VARIABLE, data=df, ax=ax, palette="viridis")


Plot saved: boxplot_water_usage_efficiency_by_crop

Phase 2: Starting data preprocessing
Saving train and test sets for reproducibility...
Data preprocessing and splitting completed. All necessary artifacts saved.

Phase 3: Starting comprehensive model evaluation and comparison
--- Evaluating model: Linear Regression ---
Predictions for Linear Regression saved to: /home/sajad/Sajad_test/Smart_Farming_Data_2024 _Regression/data_results/predictions_Linear_Regression.csv
Plot saved: evaluation_plots_Linear_Regression
--- Evaluating model: Ridge ---
Predictions for Ridge saved to: /home/sajad/Sajad_test/Smart_Farming_Data_2024 _Regression/data_results/predictions_Ridge.csv
Plot saved: evaluation_plots_Ridge
--- Evaluating model: Lasso ---
Predictions for Lasso saved to: /home/sajad/Sajad_test/Smart_Farming_Data_2024 _Regression/data_results/predictions_Lasso.csv
Plot saved: evaluation_plots_Lasso
--- Evaluating model: KNN Regressor ---
Predictions for KNN Regressor saved to: /home/sajad/Sa

/home/sajad/miniconda3/envs/sajad/lib/python3.11/site-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


Predictions for LightGBM Regressor saved to: /home/sajad/Sajad_test/Smart_Farming_Data_2024 _Regression/data_results/predictions_LightGBM_Regressor.csv
Plot saved: evaluation_plots_LightGBM_Regressor
Plot saved: models_metrics_comparison_bar_chart

Best model based on R-squared: Lasso

Phase 4: Starting hyperparameter tuning for Lasso
Hyperparameter tuning is not defined for this model.
Tuned model Lasso saved in the Best_Model_Analysis folder.

Phase 5: Starting in-depth analysis of Lasso with SHAP


  0%|          | 0/440 [00:00<?, ?it/s]

SHAP values and transformed test data saved for reproducibility.
Plot saved: shap_summary_bar_plot
Plot saved: shap_summary_dot_plot
Creating SHAP dependence plots for top features...
Plot saved: shap_dependence_plot_cat__label_watermelon
Plot saved: shap_dependence_plot_cat__label_rice
Plot saved: shap_dependence_plot_cat__label_pomegranate
Plot saved: shap_dependence_plot_cat__label_pigeonpeas
Plot saved: shap_dependence_plot_cat__label_papaya

Main script executed successfully! All results and artifacts are saved.


Starting Results Reproduction Script

Phase 0: Setting up paths and directories
Reproduction directories created successfully.
Original dataset loaded for EDA and Stats reproduction.

Phase 1: Reproducing Exploratory Data Analysis (EDA)
Table re-generated: descriptive_statistics.xlsx
Plot re-generated: target_variable_distribution


Plot re-generated: correlation_heatmap_all_features
EDA reproduction completed.

Phase 2: Reproducing Inferential Statistical Analysis
Table re-generated: anova_test_results.xlsx


/tmp/ipykernel_3914310/2102214863.py:148: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='label', y=TARGET_VARIABLE, data=df, ax=ax, palette="viridis")


Plot re-generated: boxplot_water_usage_efficiency_by_crop
Statistical Analysis reproduction completed.

Phase 3 & 4: Reproducing Model Evaluations and Comparison
--- Reproducing results for model: Linear Regression ---
  -> Metrics re-generated for Linear Regression.
Plot re-generated: evaluation_plots_Linear_Regression
--- Reproducing results for model: Ridge ---
  -> Metrics re-generated for Ridge.
Plot re-generated: evaluation_plots_Ridge
--- Reproducing results for model: Lasso ---
  -> Metrics re-generated for Lasso.
Plot re-generated: evaluation_plots_Lasso
--- Reproducing results for model: KNN Regressor ---
  -> Metrics re-generated for KNN Regressor.
Plot re-generated: evaluation_plots_KNN_Regressor
--- Reproducing results for model: SVR ---
  -> Metrics re-generated for SVR.
Plot re-generated: evaluation_plots_SVR
--- Reproducing results for model: Random Forest Regressor ---
  -> Metrics re-generated for Random Forest Regressor.
Plot re-generated: evaluation_plots_Random_For